In [ ]:
import basic
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import postprocess
import pandas as pd
import numpy as np
import run_parallel
import write_inflows
import make_wells

import matplotlib.dates as mdates
import matplotlib.cbook as cbook

import conda_scripts.utils.folium_maps as fm
from importlib import reload
import cartopy.crs as ccrs

import conda_scripts
from matplotlib import gridspec
import contextily as ctx

In [ ]:

# path = "RR_2022\Results"
remove_ponds = True
max_reach = 76
datestart = '1/1/2013'
post_process_only = True
copyfiles_from_base = True
numdays = 1095
runs = ['SWR_short_2lays_low_SWRK',
"Base_model_07142023_constant_swrk",
       "SWR_short_2lays_med_SWRK"]




In [ ]:
# path = "RR_2022\Results"
remove_ponds = True
max_reach = 76
datestart = '1/1/2013'
post_process_only = True
copyfiles_from_base = True
numdays = 3286
runs = {
   'Base': "Base_model_long_07142023_constant_swrk",
    "Low K":'SWR_long_2lays_low_SWRK',
    "Med K":'SWR_long_2lays_med_SWRK',

}

ml = basic.load_model(path = os.path.join('temp', runs[list(runs.keys())[0]]))

In [ ]:
df.plot(x = 'HEADDIFF', y = "QAQFLOW",marker = '.', linestyle =  '')

In [ ]:
os.listdir(f'versions\{runs[0]}\hydrographs')

In [ ]:
os.listdir(f'versions\{runs[0]}\\budget')

In [ ]:
reload(make_wells)

In [ ]:
import geemap
import ee
Map = geemap.Map()

# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine layers to Map
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(
    landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200}, 'Landsat 7'
)
Map.addLayer(states, {}, "US States")



label = 'Urban Growth in Las Vegas'
Map.add_landsat_ts_gif(
    label=label,
    start_year=1985,
    bands=['Red', 'Green', 'Blue'],
    font_color='white',
    frames_per_second=10,
    progress_bar_color='blue',
)


In [ ]:
del Map

In [ ]:
def load_background():
    pump = make_wells.run(name = runs[0], m = m, numdays = None, datestart = None, write_output = False)
    pump = pump.droplevel(1,0)
    
    sim_stage = pd.read_csv(os.path.join('versions',f'{runs[0]}','budget', 'stage_modeled.csv' ))
    sim_stage = sim_stage.set_index('TOTIME')
    sim_stage.index = pd.to_datetime(sim_stage.index)
    sim_stage = sim_stage.resample("1D").mean()
    sim_stage.loc[:,"Reach 76, simulated"] = sim_stage.loc[:,"Reach 76, simulated"].interpolate()
    sim_stage = sim_stage.rename(columns = {"Reach 76, simulated":'Dam Elevation'})
    sim_stage.head()
    
    rr, dry, mw, total, stg = write_inflows.run(runs[0], m= m, write_output= False)
    total = total.rename(columns = {"rrtotal":'Russian River'})
    
    return pump, sim_stage, rr, dry, mw, total, stg


def plot_bound(axisin, pump, sim_stage, total):
    
    from matplotlib.ticker import FormatStrFormatter
    
    
    sim_stage.plot.area(ax = axisin,color = 'grey',legend =False)
    axisin.plot(sim_stage.index, sim_stage.loc[:,'Dam Elevation'], c = 'blue', label = None)
    t = axisin.get_yticks()
    
    ax3 = axisin.twinx()
    ax3.set_ylabel('Russian River Discharge')
    total.plot.area(ax = ax3, color = 'lightblue', alpha = .5)
    ax3.plot(total.index, total.loc[:,'Russian River'], c = 'k', label = None)
    ax3.set_yscale('log')
    ax3.yaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax3.set_ylim([None, ax3.get_ylim()[1]*10])
    
    ax2 = axisin.twinx()
    rspine = ax2.spines['right']
    rspine.set_position(('axes', 1.15))
    ax2.set_frame_on(True)
    ax2.patch.set_visible(False)
    ax2.set_ylabel('Pumpage AF')
    pump.resample("1W").sum().div(43560.).rename(columns = lambda x: x.replace("well","Caisson ")).plot(ax = ax2, stacked = True,legend =False)

    ax2.set_ylim([0,None])
    
    axisin.set_ylim([20,150])
    axisin.set_yticks([20, 30,40])
    axisin.grid(True)
    
    h1, l1 = axisin.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    h3, l3 = ax3.get_legend_handles_labels()
    
    h1.extend(h2)
    h1.extend(h3)
    l1.extend(l2)
    l1.extend(l3)
    
    ax3.legend(h1, l1, bbox_to_anchor=(1.3, 0.), loc = 'lower left')
    
    
    


In [ ]:
pump, sim_stage, rr, dry, mw, total, stg = basic.load_background(runs[0], ml)

In [ ]:
    
fig, (ax1, ax) = plt.subplots(2,1)

plot_bound(ax, pump, sim_stage, total)
    

In [ ]:
df = pd.read_csv(f'versions\{runs["Base"]}\\budget\\sw_gw_exchange_swr.csv',index_col  =0)
df.index = pd.to_datetime(df.index)
df = -pd.concat({'Base': df}, names=['Firstlevel'],axis =1)

df2 = pd.read_csv(f'versions\{runs["Low K"]}\\budget\\sw_gw_exchange_swr.csv',index_col  =0)
df2.index = pd.to_datetime(df2.index)
df2 = -pd.concat({'Low K': df2}, names=['Firstlevel'],axis =1)

df3 = pd.read_csv(f'versions\{runs["Med K"]}\\budget\\sw_gw_exchange_swr.csv',index_col  =0)
df3.index = pd.to_datetime(df3.index)
df3 = -pd.concat({'Med K': df3}, names=['Firstlevel'],axis =1)

fig, ax = plt.subplots(1, 3,figsize = (10,5),sharey = True)
df.plot(ax = ax[0],legend = False,title = 'Base', xlabel = '')
df2.plot(ax = ax[1], legend = False,title = 'Low K', xlabel = '')
df3.droplevel(0,1).plot(ax = ax[2], legend = False,title = 'Med K', xlabel = '')
ax[2].legend(bbox_to_anchor = (1,1), loc = 'upper left')
fig.suptitle('Surface Water Groundwater Interaction Along the SWR Reaches ')
ax[0].set_ylabel(r"Flows, $ft^3/s$ (positive is inflow to stream)")

[x.grid(True)  for x in ax]
plt.savefig('streambed_sensitivity\\SWR_streambed_sens_awflow_simp_long.png', bbox_inches = 'tight')

fig, (ax, axo) = plt.subplots(2, 1,figsize = (10,10))
plot_bound(ax, pump, sim_stage, total)

df.droplevel(0,1).plot(ax = axo,legend = False,title = 'Base', xlabel = '')
axo.legend(bbox_to_anchor = (1,1), loc = 'upper left')
fig.suptitle('Surface Water Groundwater Interaction Along the SWR Reaches ')
axo.set_ylabel(r"Flows, $ft^3/s$ (positive is inflow to stream)")
axo.grid(True)


plt.savefig('streambed_sensitivity\\SWR_streambed_sens_awflow_simp_base_long.png', bbox_inches = 'tight')

In [ ]:
import postprocess
import basic
import os

In [ ]:
runs = ['SWR_long_2lays_med_SWRK',
"Base_model_long_07142023_constant_swrk",
'SWR_long_2lays_low_SWRK']

In [ ]:
for run in runs:
    ml = basic.load_model(path = os.path.join('temp',run))
    postprocess.run(run, riv_only = True, m= ml)

In [ ]:
iswr = postprocess.load_ISWRPQAQ(os.path.join('temp', runs[list(runs.keys())[2]], 'Results'), True, 76, '1/1/2014')

In [ ]:
iswr.droplevel([1,2,3,4,6]).query("REACH>60").loc[:,'DEPTH'].groupby(level = [0,1]).mean().unstack().iloc[:,::3].plot()

In [ ]:
iswr.droplevel([1,2,3,4,6]).query("REACH==60 or REACH==76").loc[:,'DEPTH'].groupby(level = [0,1]).mean().unstack().plot()

In [ ]:
sfr_rout = gpd.read_file("SFR_files\only_sfr_routing.shp")
sfr_rout.plot()

In [ ]:
iswr.droplevel([1,2,3,4,5,6]).loc[:,'QAQFLOW'].plot()

In [ ]:
sfr = ml.SFR

In [ ]:
pd.DataFrame(sfr.reach_data).query("iseg == 51")

In [ ]:
pd.DataFrame(sfr.reach_data).query("iseg == 55")

In [ ]:
dfall = df2.join(df).join(df3).stack('Firstlevel').swaplevel()

In [ ]:
fig, ax = plt.subplots(8, sharex = True,figsize = (8,8))

for n, col in enumerate(dfall.columns):
    
    dfall.loc[:,col].unstack(0).plot(ax = ax[n],legend = False)
    ax[n].set_ylim([-3, 2.5])
    ax[n].grid(True)
    ax[n].text(1,0,col, transform = ax[n].transAxes)
    
ax[0].legend(bbox_to_anchor = (1,1.5), loc = 'upper left')


ax[n].xaxis.set_minor_locator(mdates.MonthLocator(interval = 1))
ax[n].xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7  ]))
ax[n].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(mdates.MonthLocator(bymonth=[1, 7,  ])))

ax[4].set_ylabel(r"Average Flows, $ft^3/s$ (positive is inflow to stream)")

plt.savefig('streambed_sensitivity\\SWR_streambed_sens_aq_flow_long.png', bbox_inches = 'tight')

In [ ]:
wells = gpd.read_file("GIS/wells_mod.shp")

In [ ]:
m =  basic.load_model()



In [ ]:
def load_h(run, name):
    allmeas = pd.read_csv(os.path.join('versions',run, 'hydrographs','all_meas.csv'),index_col = 0)
    # allmeas = pd.concat({name: df}, names=['Firstlevel'],axis =1)
    allmeas.index = pd.to_datetime(allmeas.index)
    return allmeas

df = load_h(runs['Low K'], 'Low K')
df1 = load_h(runs['Base'], 'Base')
df2 = load_h(runs['Med K'], 'Med K')

nrow = 2
ncol = 4
fig, ax = plt.subplots(nrow,ncol,sharex = True,figsize = (30,8) )

ax = ax.ravel()

def plot_well(welldf, col = 'Simulated', color = None, label = None):
    c = 0
    for name, well in welldf.loc[welldf.station_no.isin([f'Caisson{x}' for x in np.arange(10)])].groupby('station_no'):
        ax[c].plot(well.index, well.loc[:,col], color = color, label = label)

        c+=1

plot_well(df1,col = 'Observed', color ='grey',label = 'Observed')
plot_well(df1,  label = 'Base')
plot_well(df2, label = 'Med K')
plot_well(df, label = "Low K")
   

c = 0
for name, well in df.loc[df.station_no.isin([f'Caisson{x}' for x in np.arange(10)])].groupby('station_no'):
    ax[c].grid(True)
    ax[c].text(1,1,name, ha = 'right',transform = ax[c].transAxes)
    c+=1
    
plot_bound(ax[-1], pump, sim_stage, total)
ax.reshape((nrow,ncol))[0,-1].legend(loc = 'upper left', bbox_to_anchor  = (1,1))
ax[-2].yaxis.set_visible(False)
fig.suptitle('Simulated Heads for Caissons - Comparing Streambed K versus Base K')
# ax[2].xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1,3, 6, 9, ]))
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
# ax[5].xaxis.set_major_formatter(
#     mdates.ConciseDateFormatter(mdates.MonthLocator(bymonth=[1,3, 6, 9, ])))

plt.savefig('streambed_sensitivity\\SWR_streambed_sens_long.png', bbox_inches = 'tight')

In [ ]:
caisson

In [ ]:
def load_h(run, name):
    allmeas = pd.read_csv(os.path.join('versions',run, 'hydrographs','all_meas.csv'),index_col = 0)
    # allmeas = pd.concat({name: df}, names=['Firstlevel'],axis =1)
    allmeas.index = pd.to_datetime(allmeas.index)
    return allmeas

def load_h(run, name):
    allmeas = pd.read_csv(os.path.join('versions',run, 'hydrographs','all_meas.csv'),index_col = 0)
    # allmeas = pd.concat({name: df}, names=['Firstlevel'],axis =1)
    allmeas.index = pd.to_datetime(allmeas.index)
    return allmeas

def plot_well_indivdual(welldf, col = 'Simulated', color = None, label = None, well = None):

    welldf = welldf.loc[welldf.station_no==well]
    ax[0].plot(welldf.index, welldf.loc[:,col], color = color, label = label)


            
df = load_h(runs['Low K'], 'Low K')
df1 = load_h(runs['Base'], 'Base')
df2 = load_h(runs['Med K'], 'Med K')

nrow = 2
ncol = 1

for caisson in [f'Caisson{x}' for x in np.arange(1,7)]:
    fig, ax = plt.subplots(nrow,ncol,sharex = True,figsize = (12,12) )
    ax = ax.ravel()


    plot_well_indivdual(df1,col = 'Observed', color ='grey',label = 'Observed', well = caisson)
    plot_well_indivdual(df1,  label = "Base", well = caisson)
    plot_well_indivdual(df2, label = "Med K", well = caisson)
    plot_well_indivdual(df, label = "Low K", well = caisson)


    c = 0

    ax[c].grid(True)
    ax[c].text(1,1,caisson, ha = 'right',transform = ax[c].transAxes)

    plot_bound(ax[-1], pump, sim_stage, total)
    ax.reshape((nrow,ncol))[0,-1].legend(loc = 'upper left', bbox_to_anchor  = (1,1))
    # ax[-2].yaxis.set_visible(False)
    fig.suptitle(f'Simulated Heads for {caisson}  - Comparing Streambed K versus Base K')

    
    plt.savefig(f'streambed_sensitivity\\SWR_streambed_sens_{caisson}_long.png', bbox_inches = 'tight')

In [ ]:


df = load_h(runs['Low K'], 'Low K')
df1 = load_h(runs['Base'], 'Base')
df2 = load_h(runs['Med K'], 'Med K')

nrows,ncols = 4,5
fig, ax = plt.subplots(nrows,ncols,sharex = True,figsize = (15,8) )

ax = ax.ravel()

def plot_well(welldf, col = 'Simulated', color = None, label = None):
    c = 0
    for name, well in welldf.loc[~welldf.station_no.isin([f'Caisson{x}' for x in np.arange(10)])].groupby('station_no'):
        ax[c].plot(well.index, well.loc[:,col], color = color, label = label)

        c+=1

plot_well(df1,col = 'Observed', color ='grey',label = 'Observed')
plot_well(df1,  label = 'Base')
plot_well(df2, label = 'Med K')
plot_well(df, label = "Low K")
   

c = 0
for name, well in df.loc[~df.station_no.isin([f'Caisson{x}' for x in np.arange(10)])].groupby('station_no'):
    ax[c].grid(True)
    ax[c].text(1,1,name, ha = 'right',transform = ax[c].transAxes)
    c+=1
    
ax.reshape((nrows,ncols))[0,-1].legend(loc = 'upper left', bbox_to_anchor  = (1,1))
  
fig.suptitle('Simulated Heads for Monitoring Wells - Comparing Streambed K versus Base K')

ax[2].xaxis.set_minor_locator(mdates.MonthLocator(interval = 6))
ax[2].xaxis.set_major_locator(mdates.YearLocator(2,1,1))
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax[5].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(mdates.MonthLocator(bymonth=[1,  ])))

plt.savefig('streambed_sensitivity\\SWR_streambed_sens_MW_long.png', bbox_inches = 'tight')

In [ ]:


gs = gridspec.GridSpec(2, 1, height_ratios = [2,1.5])
fig = plt.figure(figsize = (10,10))
axsRight = fig.add_subplot(gs[0], projection=ccrs.epsg(2226))
# axsRight = subfigs[1].subplots(1, 1, sharex=True)
# fig, axsRight = basic.basic_map(m, True,ax = axsRight, fig = subfigs[1] , maptype='ctx.USGS.USTopo')
fig, ax = basic.basic_map(ml, add_basemap=True, ax = axsRight, fig = subfigs[1] , maptype='ctx.USGS.USTopo')
ctx.add_basemap(ax, crs = 2226, source = "USGS.USTopo")
wells.plot(ax = ax, edgecolor = 'k')

# swrpt['geometry'] = swrpt['geometry'].centroid
# set_bounds_to_shape(ax, swr)
basic.set_bounds(ax)
# text = ph.label_points(ax, swrpt.loc[swrpt.rno%5==0].reset_index(),
#                        'rno', basin_name = None,text_color = 'k' )

text1 = conda_scripts.plot_help.label_points(ax, wells,
                                             
                       'station_na', iter_limit = 10000, basin_name = None, already_str = False, fmt = "s",add_box = True,
                                             masked_area = None,
                       arrow_props =  dict(arrowstyle='-', color='grey'))

In [ ]:


ma = wells.explore()
fm.add_layers(ma)
ma

In [ ]:

reload(basic)

In [ ]:
area1 = {"Mirabel":['LRR0134', "LRR0132", "LRR0127", "LRR0153"]}
area2 = {"Mark Wesk":['LRR0142', "LRR0143"]}
area3 = {"Lower":['LRR0140', "LRR0061", "LRR0062", "LRR0063",'LRR0064', "LRR0065", ]}
area4 = {"Upper":["LRR0068", "LRR0125", "LRR0131", "LRR0124"]}
area5 = {"Wohler":['LRR0070', "LRR0074", "LRR0082", "LRR0076"]}

def fix(x):
    
    f = pd.DataFrame({'well':list(x.values())[0], 'area': list(x.keys())*len(list(x.values())[0]) }, )
    
    return f

wellnames = pd.concat([fix(xi) for xi in [area1, area2, area3, area4, area5, ]])

def plot_well_by_area(welldf,ax,  col = 'Simulated', color = None, label = None,area = None ):
    welldf = welldf.loc[welldf.well.isin(area.well)]
    c = 0
    for name, well in welldf.groupby('well'):
        ax[c].plot(well.index, well.loc[:,col], color = color, label = label)        
        ax[c].text(1,1,name, ha = 'right',transform = ax[c].transAxes)
        
        ax[c].grid(True)
        
        c+=1
        
for area, group in wellnames.groupby('area'):
    print(f"area:  {area}\ngroup:\n{group}\n")
    fig = plt.figure(layout='constrained', figsize=(20, 10))
    fig.suptitle('Simulate Waterlevel Sensitivity to Varying Streambed K', fontsize='xx-large')
    subfigs = fig.subfigures(1, 2, wspace=0.0001, width_ratios=[1, 1])
    
    print(f'group shape before {group.shape[0]}')
    group = group.loc[group.well.isin(df.well)]
    print(f'group shape after {group.shape[0]}')
    ncol = int(np.ceil(group.shape[0]/3))
    if ncol==1:
        nrow = group.shape[0]
    else:
        nrow = 3
        
    axsLeft = subfigs[0].subplots(nrow, ncol, sharey=True,sharex = True, squeeze = False)
    axsLeft = axsLeft.ravel()

    plot_well_by_area(df1,axsLeft, col = 'Observed', color ='grey',label = 'Observed', area = group)
    plot_well_by_area(df1, axsLeft, label = "Base", area = group)
    plot_well_by_area(df2,axsLeft, label = "Med K", area = group)
    plot_well_by_area(df,axsLeft, label = "Low K", area = group)
    
    # axsLeft.reshape((nrow,ncol))[0,-1].legend(loc = 'upper left', bbox_to_anchor  = (1,1))
    
    axsLeft[0].xaxis.set_minor_locator(mdates.MonthLocator(interval = 6))
    axsLeft[0].xaxis.set_major_locator(mdates.YearLocator(2,1,1))
    # ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    axsLeft[0].xaxis.set_major_formatter(
        mdates.ConciseDateFormatter(mdates.MonthLocator(bymonth=[1,  ])))
    
    
    #aright side
    from matplotlib import gridspec
    gs = gridspec.GridSpec(2, 1, height_ratios = [2,1.5])
    axsRight = subfigs[1].add_subplot(gs[0], projection=ccrs.epsg(2226))
    # axsRight = subfigs[1].subplots(1, 1, sharex=True)
    fig, axsRight = basic.basic_map(ml, False,ax = axsRight, fig = subfigs[1])
    ctx.add_basemap(axsRight, crs = 2226, source = "USGS.USTopo")
    wells.plot(ax = axsRight, edgecolor = 'g', markersize = 3)
    wells.loc[wells.station_na.isin(group.well)].plot(ax = axsRight, edgecolor = 'k')

    basic.set_bounds(axsRight)
    text1 = conda_scripts.plot_help.label_points(axsRight, wells.loc[wells.station_na.isin(group.well)],

                           'station_na', iter_limit = 10000, basin_name = None, already_str = False, fmt = "s",add_box = True,
                                                 masked_area = None,
                           arrow_props =  dict(arrowstyle='-', color='black'))

    # subfigs[1].set_facecolor('0.85')
    # subfigs[1].colorbar(pc, shrink=0.6, ax=axsRight)
    # subfigs[1].suptitle('Right plots', fontsize='x-large')
    
    axsRight.set_title(area+' Area', fontsize='xx-large')
    # fig.suptitle(area+' Area', fontsize='xx-large')
     # gridspec_kw={'height_ratios': [1, 1, 3
    axs_bot = subfigs[1].add_subplot(gs[1],)
    plot_bound(axs_bot, pump, sim_stage, total)
    
    plt.savefig(f'streambed_sensitivity\\SWR_streambed_sens_MW_{area}_long.png', bbox_inches = 'tight')
    
    

In [ ]:
dfall.unstack(0).sum().unstack(1).plot.bar()
fig.suptitle('Simulated Heads for Monitoring Wells - Comparing Streambed K versus Base K')
plt.savefig('streambed_sensitivity\\SWR_streambed_sens_MW_long.png', bbox_inches = 'tight')